# Bayesian Information Criterion

In [ ]:
import os
import pandas as pd
import glob
import math
import emcee
import string
import corner
import numpy as np
from tqdm import tqdm
from numba import jit
import concurrent.futures
from multiprocessing import Pool
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
from scipy.integrate import odeint
import seaborn as sns
import scipy.stats as stats
import sys

# Find and sort all text files in the ./data directory
data_files = sorted(glob.glob("./data/*.txt"))

# Dictionary to store the parsed data, grouped by data type
full_data = {}

# Loop through each file in the data_files list
for file in data_files:
    # Extract the data type (treatment and cell type) from the file name
    data_type = os.path.basename(file).split('/')[-1].split('_c')[0]
    
    # Extract the mouse name (cohort identifier) from the file name
    mouse_name = 'c' + file.split('/')[-1].split('_c')[-1].split('_t')[0]
    
    # Extract the treatment days from the file name (as integers)
    t_days = np.array([int(t) for t in file.split('/')[-1].split('_c')[-1].split('.txt')[0].split('_t')[1:]])
    
    # Load the measurement data from the file (time, tumor volume)
    data = np.loadtxt(file)
    
    # Extract the specific treatment from the data type (e.g., radiation or control)
    treatment = os.path.basename(data_type).split('_')[0]
    
    # If this data type hasn't been seen before, create a new list in full_data
    if data_type not in full_data:
        full_data[data_type] = []
    
    # Append the mouse's data (including name, tumor measurements, treatment, and treatment days) to the appropriate group
    full_data[data_type].append({
        'name': mouse_name,
        'data': data,
        'treatment': treatment,
        'treatment_days': t_days
    })
    
def calculate_n_total(full_data, group):
    n_total = 0
    for mouse_data in full_data[group]:
        n_total += len(mouse_data['data'])
    return n_total

all_models = [
                '_LQ',
                '_ExponentialDecay',                               
                '_CumulativeDose', 
                '_AccelED', 
                '_AccelCD',
                '_CDwithLogFactor', 
                '_CDwithExpFactor', 
                '_CDwithSinFactor', 
                '_EDwithLogFactor', 
                '_EDwithExpFactor', 
                '_EDwithSinFactor',
                "_ExponentialDecayDelay",
                "_EDEFDelay",
                "_CumulativeDoseDelay",
                "_CDEFDelay",
                '_ACDDelay',
                '_AEDDelay',
                '_CDSFDelay',
                '_EDSFDelay'         
 ]


group_ll_total = {'radiation_sensitive': {}, 'radiation_resistant': {}}
group_params_total = {'radiation_sensitive': {}, 'radiation_resistant': {}}
group_obs_total = {'radiation_sensitive': 0, 'radiation_resistant': 0}

for model in all_models:
    for group in group_ll_total:
        group_ll_total[group][model] = 0
        group_params_total[group][model] = 0

for group in ['radiation_sensitive', 'radiation_resistant']:
    n_total = calculate_n_total(full_data, group)
    group_obs_total[group] = n_total 

    for model in all_models:

        files_location = f'./Output_Calibration/multi_ll_pars_{group}{model}.npz'
        npzfile = np.load(files_location)

        max_ll = npzfile['max_ll']
        theta = npzfile['pars']
        k = len(theta)

        group_ll_total[group][model] = max_ll
        group_params_total[group][model] = k

header = "group & " + " & ".join([model.replace('_', '') for model in all_models]) + " \\\\ \\hline"
print(header)

for group in ['radiation_sensitive', 'radiation_resistant']:
    row_text = f"{group} " 

    n_total = group_obs_total[group]

    for model in all_models:
        total_ll = group_ll_total[group][model]
        total_params = group_params_total[group][model]

        if n_total > 0:
            bic_global = total_params * np.log(n_total) - 2 * total_ll
        else:
            bic_global = np.nan

        row_text += f"& {bic_global:.2f} "

    print(row_text + "\\\\ \\hline")

# BIC weights

In [3]:

bic_weights = {'radiation_sensitive': {}, 'radiation_resistant': {}}

for group in ['radiation_sensitive', 'radiation_resistant']:

    bic_values = []

    for model in all_models:
        total_ll = group_ll_total[group][model]
        total_params = group_params_total[group][model]
        n_total = group_obs_total[group]

        if n_total > 0:
            bic_global = total_params * np.log(n_total) - 2 * total_ll
        else:
            bic_global = np.nan 

        bic_values.append(bic_global)

    bic_min = min(bic_values)

    delta_bic = [bic - bic_min for bic in bic_values]
    weights = [np.exp(-d_bic / 2) for d_bic in delta_bic]
    sum_weights = sum(weights)
    normalized_weights = [w / sum_weights for w in weights]

    for i, model in enumerate(all_models):
        bic_weights[group][model] = normalized_weights[i]

header = "group & " + " & ".join([model.replace('_', '') for model in all_models]) + " \\\\ \\hline"
print(header)

for group in ['radiation_sensitive', 'radiation_resistant']:
    row_text = f"{group} "
    for model in all_models:
        row_text += f"& {bic_weights[group][model]:.4f} "
    print(row_text + "\\\\ \\hline")


group & ExponentialDecayDelay & EDEFDelay & CumulativeDoseDelay & CDEFDelay & ACDDelay & AEDDelay & CDSFDelay & EDSFDelay \\ \hline
radiation_sensitive & 0.0000 & 1.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 \\ \hline
radiation_resistant & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0000 & 0.0031 & 0.9969 & 0.0000 \\ \hline
